In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
QiskitRuntimeService.save_account(
  token="", # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
  name="QuantumTut", # Optional
  instance="crn:v1:bluemix:public:quantum-computing:us-east:a/5f61ff06ea3f49e59b409ea9e841caef:45ca65d9-2b04-4db0-bb79-3cf0beb35656::", # Optional
  set_as_default=True, # Optional
  overwrite=True, # Optional
)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
# run every time you need the service
service = QiskitRuntimeService(name="QuantumTut")

# Variational Quantum Classifier (QNN + NeuralNetworkClassifier)

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator

from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import NeuralNetworkClassifier

# ---------- 1. Dataset ----------
X, y = make_moons(n_samples=200, noise=0.15, random_state=42)
# Scale features to [0, π] for angle encoding
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

num_features = X_train.shape[1]
num_qubits = num_features  # simple choice

# ---------- 2. Build the quantum model: feature map + ansatz ----------
feature_map = ZZFeatureMap(
    feature_dimension=num_features,
    reps=1,
    entanglement="full",
)
ansatz = RealAmplitudes(
    num_qubits,
    reps=1,
    entanglement="full",
)

qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

# ---------- 3. Wrap in an EstimatorQNN ----------
estimator = Estimator()

qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator,
)

# ---------- 4. Make a scikit-learn style classifier ----------
optimizer = COBYLA(maxiter=100)

classifier = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    one_hot=False,   # binary labels 0/1
)

# ---------- 5. Train & evaluate ----------
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Variational quantum classifier accuracy: {acc:.3f}")


# Quantum Kernel + QSVC (Quantum SVM style classifier)

In [ ]:
import numpy as np
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StateFidelity
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# ---------- 1. Toy dataset (non-linear) ----------
X, y = make_circles(n_samples=200, noise=0.1, factor=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# ---------- 2. Define quantum feature map & kernel ----------
num_features = X_train.shape[1]

feature_map = ZZFeatureMap(
    feature_dimension=num_features,
    reps=2,
    entanglement="full",
)

fidelity = StateFidelity()  # computes ⟨ψ(x)|ψ(x')⟩
quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
    fidelity=fidelity,
)

# ---------- 3. Quantum SVM (QSVC) ----------
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train, y_train)

# ---------- 4. Evaluate ----------
y_pred = qsvc.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"QSVC test accuracy: {acc:.3f}")


# QAOA for Max-Cut (small graph, Qiskit Optimization + QAOA)

In [ ]:
import numpy as np

from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA

from qiskit_optimization.applications import Maxcut
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# ---------- 1. Define a Max-Cut instance (4-node graph) ----------
# Adjacency matrix of an unweighted graph (0 = no edge, 1 = edge)
w = np.array([
    [0, 1, 1, 1],
    [1, 0, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 0],
], dtype=float)

maxcut = Maxcut(w)
qp = maxcut.to_quadratic_program()
print("Quadratic program:")
print(qp.prettyprint())

# ---------- 2. Solve classically as a reference ----------
exact_mes = NumPyMinimumEigensolver()
exact_meo = MinimumEigenOptimizer(exact_mes)
exact_result = exact_meo.solve(qp)
print("\n[Classical reference]")
print(exact_result.prettyprint())

# ---------- 3. Define QAOA ----------
sampler = Sampler()  # statevector-based sampler by default
optimizer = COBYLA(maxiter=100)

qaoa_mes = QAOA(
    sampler=sampler,
    optimizer=optimizer,
    reps=2,         # number of QAOA layers p
)

qaoa_meo = MinimumEigenOptimizer(qaoa_mes)

# ---------- 4. Run QAOA ----------
qaoa_result = qaoa_meo.solve(qp)

print("\n[QAOA result]")
print(qaoa_result.prettyprint())

# Cut value & bitstring
print("Optimal bitstring (QAOA):", qaoa_result.x)
print("Max-Cut objective value:", qaoa_result.fval)
